## Final Report 

### Introduction
A research group in CS at UBC is running a Minecraft server called Plaicraft to learn about video game players’ habits. Minecraft is a popular survival and building game that can be played alone or with others. A Minecraft server is a place where players can play the game together. Plaicraft (UBC CS’ server) is free to play, but Minecraft (the game) costs money, and using the Plaicraft server for research means it's much more resource-intensive than a normal Minecraft server. So, it will be very helpful to know when resources are needed. 

#### Question
The question we are answering is: When do the most players tend to play? 
We will answer this question in two steps. First, we will find out how activity varies throughout the hours of the day. Second, we will find out if weekday and weekend activity levels differ.

#### Dataset Description 
We used sessions.csv to answer our question 
sessions.csv is data collected through the server, it has 1535 observations and 5 variables, which are:
- hashedemail, an anonymized email of the player,
- start time, the date and time of when the session started as an object,
- end time, the date and time of when the session ended as an object,
- original start time, a float representing the start time in UNIX time, as in seconds since 1/1/1970,
- original end time, the same format as original start time, but for end time.

### Methods and Results
Our plan is to use KNN regression to determine how many players will be online at a given time on a given day. Our response variable is the number of concurrent players, and our predictor variable is the hour of the day. To account for outliers, we will find the maximum concurrent sessions for each day of the week.

Given the qualitative nature of the day of the week variable, we will perform two analyses. One analysis will not account for the day of week, and the second analysis will be a comparison between two analyses of weekday sessions and weekend sessions. 

Before we started our analysis, we created some exploratory visualizations to give us a better understanding of the data. We looked at total sessions by hour, total active sessions by day, and total active sessions by hour by day.

#### Wrangling Data: 
We will convert the start and end times to datetime format. From the orignal_start_time and original_end_time columns, we can tell that these times are in GMT. Because we are in PST, and we are assuming most players are playing in PST, we will convert all the timestamps to PST. Then, we will find the day of week of each session. Finally, we will find out the amount of concurrent players for each hour of each day of the week with a date range and a for loop for each hour to determine how many sessions' start times are before the hour and end times are after the hour. At the wrangling stage, we will create three frames: one with all the data, one with only weekday sessions data, and one with only weekend sessions data. 

The dataframe with all the sessions, named concurrent_sessions, has the following columns:
- time, as a datetime timestamp
- active_sessions, an integer of how many active sessions there are
- day_of_week, which day of the week the observation took place
- hour, the hour of the day the observation took place

The other two dataframes, our weekday_sessions and weekend_sessions have the same columns as concurrent_sessions, with the addition of the day_or_end column, which is a string representation of whether the observation is a weekday or weekend.

#### Analysis

For each iteration of our analysis, we created a new model to make sure the results are accurate to the specific data. For our models, we first split our data into training and testing sets so we can evaluate our model's efficacy. We created a pipeline to scale the data and apply the KNN regression model. Then, we trained our model on the training set, tuned it using 5 fold cross-validation to find the best k, and then tested it on the testing set. We are satisfied with our models, their RMSPE were all under 1, which is good because we are dealing with active sessions in units of one active session. Then, we made predictions for each of our three datasets. 

#### Results

We first created a chart to show the maximum concurrent sessions recorded for each day of the week. Even though these numbers are very different from the rest of our predictions, we think it's important for the Plaicraft team to know about outlier scenarios to ensure they can support the activity level. 

We presented our regression results in the form of a layered chart. The first layer is a scatterplot of the active sessions by hour, with the size of the points representing how common that level of activity is. It's more informative to see where the points vary in size, because our observations about active sessions heavily overlap. The second layer is a line of our predicted number of active sessions. 

For our weekday and weekend comparisons, we first concatenated the two dataframes. Then, we visualized them in the same way as our general predictions, but faceted them by weekday and weekend. 

However, we noticed that the size of points between the scatterplots was different, because there are fewer weekend days than weekday days, hence, there were fewer observations on weekends. So, we created bar charts comparing predicted and recorded weekday and weekend average activity levels to understand the results better. 


### Conclusion and Discussion

#### Findings 
We found that: 
- Most active sessions take place in the afternoons and evenings, fewer take place during typical work or school hours, and even fewer take place late at night. 
- During the day, weekend sessions are more popular. During the evening, weekday sessions are more popular. 
- The maximum number of concurrent sessions was 7, and it occurred on a Saturday.

#### Our expectations 
Before conducting our analysis, we predicted that:
- most sessions would take place in the afternoons and evenings when most people have free time
- there would be fewer sessions at night when most people sleep, and during normal work and school hours
- there would be more activity on the weekends because less people have school and work. 

Our conclusions are mostly consistent with our predictions, but we were surprised to find the differences between weekdays and weekends when it comes to the most popular time of day. 

#### Impact 
Knowing when most players are online can help the Plaicraft organizers manage their licences. These findings can tell them when they will need the most licenses, and how many licenses they expect to use. 

#### Potential Issues 
We're hesitant to say our findings are impactful due to how small the project is, how small dataset is, and the potentially biased way it was collected. 

If we are looking for information specifically about the Plaicraft server for the next few months, our results are okay. However, if we are looking for information about video game habits in general, our results are not very impactful. 

There were usually very few active sessions at any given time. The dataset only has 1535 observations, and they were collected in the span of only five months, from April to September of 2024. Additionally, the player demographics and activity are not representative of video games in general. Seeing as the DSCI100 students' grades are affected by their playtime, we are highly incentivized to play on the server. So, their behaviour is not representative of average video game players. There might be players who otherwise would not play, people "cramming" their playtime requirement as they would an assignment, or people who play exactly three hours to fulfill the requirement. Since some data comes from people treating Plaicraft like an assignment, the results drawn from this data set may not apply to actual habits of people playing games for leisure. 

#### Future Questions
Some other questions that can help us learn about player habits and the Plaicraft server: 
- Is the activity increasing over time?
- Are there differences in activity during school breaks, such as reading breaks, winter holidays, and the summer months?
- Are there differences in activity on major holidays such as Christmas and Thanksgiving? 
- Do player demographics, such as age, affect when players are active?
- Does the length of a session vary depending on the time of day, day of the week, or player demographics?


In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pytz

In [ ]:
#loading data and converting sessions start and end times to datetime format 
sessions = pd.read_csv('sessions.csv')
sessions['start_time'] = pd.to_datetime(sessions['start_time'], format='%d/%m/%Y %H:%M')
sessions['end_time'] = pd.to_datetime(sessions['end_time'], format='%d/%m/%Y %H:%M')

In [ ]:
gmt = pytz.timezone('GMT')
pst = pytz.timezone('US/Pacific')

sessions['start_time'] = sessions['start_time'].dt.tz_localize(gmt).dt.tz_convert(pst)
sessions['end_time'] = sessions['end_time'].dt.tz_localize(gmt).dt.tz_convert(pst)

sessions

## Exploratory visualizations to help us understand the data before analysis:

In [ ]:
#chart to show concurrent sessions by hour
total_sessions_chart = alt.Chart(concurrent_sessions, title = 'Total Active Sessions by Hour').mark_bar().encode(
    x = alt.X('hour:O').title('Hour of Day'),
    y = alt.Y('active_sessions').title('Total Active Sessions')
)

total_sessions_chart

In [ ]:
dayofweek_exploratory_plot = alt.Chart(concurrent_sessions, title = 'Total Sessions by Day of Week').mark_bar().encode(
    x = alt.X('day_of_week', 
              sort = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], 
              axis=alt.Axis(labelAngle=-45))
    .title("Day of Week"),
    y = alt.Y ('active_sessions').title('Total Sessions')
)
dayofweek_exploratory_plot

In [ ]:
#charts to show concurrent sessions by hour and day of week
total_sessions_dayofweek = alt.Chart(concurrent_sessions, title = 'Total Active Sessions by Hour by Day').mark_bar().encode(
    x = alt.X('hour:O').title('Hour of Day'),
    y = alt.Y('active_sessions').title('Total Active sessions')
).facet(
    row=alt.Row('day_of_week:N', title='Day of the Week', sort=[
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    ]))
total_sessions_dayofweek

## Our analysis:

In [ ]:
#creating a range 
hours = pd.date_range(start=sessions['start_time'].min().floor('h'), 
                      end=sessions['end_time'].max().ceil('h'), freq='h')

concurrent_sessions = []

#a loop that determines how many sessions start before the hour and end after the hour
for h in hours:
    active_players = ((sessions['start_time'] <= h) & (sessions['end_time'] >= h)).sum()
    concurrent_sessions.append({'time': h, 'active_sessions': active_players})


concurrent_sessions = pd.DataFrame(concurrent_sessions)
concurrent_sessions = concurrent_sessions.assign(day_of_week = concurrent_sessions['time'].dt.day_name(), hour = concurrent_sessions['time'].dt.hour.astype(int))

concurrent_sessions

In [ ]:
weekday_sessions = concurrent_sessions[(
    concurrent_sessions['day_of_week']=='Monday')
|(concurrent_sessions['day_of_week']=='Tuesday')
|(concurrent_sessions['day_of_week']=='Wednesday')
|(concurrent_sessions['day_of_week']=='Thursday')
|(concurrent_sessions['day_of_week']=='Friday')
]
weekday_sessions = weekday_sessions.assign(day_or_end = 'weekday')

weekend_sessions = concurrent_sessions[(
    concurrent_sessions['day_of_week']=='Saturday')
|(concurrent_sessions['day_of_week']=='Sunday')]

weekend_sessions = weekend_sessions.assign(day_or_end = 'weekend')

### Analysis by hour, regardless of day of week

In [ ]:
##train test split 
sessions_train, sessions_test = train_test_split(
    concurrent_sessions,
    test_size=0.25,
    random_state=2000,  # Do not change the random_state
)
X_train = sessions_train[['hour']]  # A single column data frame
y_train = sessions_train['active_sessions']  # A series

X_test = sessions_test[['hour']]  # A single column data frame
y_test = sessions_test['active_sessions']  # A series

In [ ]:
#pipeline and tuning
sessions_pipe = make_pipeline(StandardScaler(), KNeighborsRegressor())

param_grid = {
    "kneighborsregressor__n_neighbors": range(1, 50, 1),
}

sessions_tuned = GridSearchCV(sessions_pipe, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
sessions_results = pd.DataFrame(sessions_tuned.fit(X_train, y_train).cv_results_) 

sessions_tuned.fit(
    sessions_train[['hour']],  # A single-column data frame
    sessions_train['active_sessions']  # A series
)

In [ ]:
sessions_tuned.best_params_

In [ ]:
sessions_best_RMSPE = -sessions_tuned.best_score_
sessions_best_RMSPE

### The RMSPE is not too different between the training and testing sets, and it is not bad considering we are working in units of individual licenses. 

In [ ]:
sessions_test["predicted"] = sessions_tuned.predict(sessions_test)
RMSPE = mean_squared_error(
    y_true=sessions_test["active_sessions"],
    y_pred=sessions_test["predicted"]
)**(1/2)
RMSPE

In [ ]:
sessions_prediction = sessions_tuned.predict(X_test)
sessions_summary = mean_squared_error(sessions_test['hour'], sessions_prediction)**(1/2)

sessions_preds = sessions_train.assign(
    predictions = sessions_tuned.predict(sessions_train[['hour']])
)
sessions_preds

### Analysis by hour, seperating between weekdays and weekends

In [ ]:
#weekdays
weekday_sessions_train, weekday_sessions_test = train_test_split(
    weekday_sessions,
    test_size=0.25,
    random_state=2000,  # Do not change the random_state
)
weekday_X_train = weekday_sessions_train[['hour']]  # A single column data frame
weekday_y_train = weekday_sessions_train['active_sessions']  # A series

weekday_X_test = weekday_sessions_test[['hour']]  # A single column data frame
weekday_y_test = weekday_sessions_test['active_sessions']  # A series

#preprocessor and model same as before
weekday_sessions_tuned = GridSearchCV(sessions_pipe, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
weekday_sessions_results = pd.DataFrame(sessions_tuned.fit(weekday_X_train, weekday_y_train).cv_results_) 
weekday_sessions_tuned.fit(
    weekday_sessions_train[['hour']],  # A single-column data frame
    weekday_sessions_train['active_sessions'])  # A series

weekday_sessions_prediction = weekday_sessions_tuned.predict(weekday_X_test)
weekday_sessions_preds = weekday_sessions_train.assign(
    predictions = weekday_sessions_tuned.predict(weekday_sessions_train[['hour']])
)
weekday_sessions_preds

In [ ]:
weekday_best_RMSPE = -weekday_sessions_tuned.best_score_
weekday_best_RMSPE

In [ ]:
#weekends
weekend_sessions_train, weekend_sessions_test = train_test_split(
    weekend_sessions,
    test_size=0.25,
    random_state=2000,  # Do not change the random_state
)
weekend_X_train = weekend_sessions_train[['hour']]  # A single column data frame
weekend_y_train = weekend_sessions_train['active_sessions']  # A series

weekend_X_test = weekend_sessions_test[['hour']]  # A single column data frame
weekend_y_test = weekend_sessions_test['active_sessions']  # A series

#preprocessor and model same as before
weekend_sessions_tuned = GridSearchCV(sessions_pipe, param_grid, cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
weekend_sessions_results = pd.DataFrame(sessions_tuned.fit(weekend_X_train, weekend_y_train).cv_results_) 
weekend_sessions_tuned.fit(
    weekend_sessions_train[['hour']],  # A single-column data frame
    weekend_sessions_train['active_sessions'])  # A series

weekend_sessions_prediction = weekend_sessions_tuned.predict(weekend_X_test)
weekend_sessions_preds = weekend_sessions_train.assign(
    predictions = weekend_sessions_tuned.predict(weekend_sessions_train[['hour']])
)
weekend_sessions_preds

In [ ]:
weekend_best_RMSPE = -weekend_sessions_tuned.best_score_
weekend_best_RMSPE

These RMSPE are okay because we are working with units of individual licenses.

In [ ]:
#combining the weekday and weekend predictions for easy comparisons 
dayofweek_preds = pd.concat([weekday_sessions_preds , weekend_sessions_preds])
dayofweek_preds

## Plots visualizing our results:

In [ ]:
max_sessions_plot = alt.Chart(concurrent_sessions, title = 'Maximum Concurrent Sessions by Day of Week').mark_bar().encode(
    x = alt.X('day_of_week', sort = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], axis=alt.Axis(labelAngle=-45))
    .title("Day of Week"),
    y = alt.Y ('max(active_sessions)').title('Maximum Active Sessions Recorded')
)
max_sessions_plot

This plot tells us the maximum number of concurrent sessions that has occured, by day of the week. From this plot, we know that plaicraft should have seven licenses to account for the absolute maximum.

In [ ]:
sessions_data_plot = alt.Chart(sessions_preds).mark_point(opacity = 0.4, filled = True).encode(
    x = alt.X('hour').title('Hour of Day'),
    y = alt.Y('active_sessions').title('Active Sessions'),
    size = 'count():Q'
)

sessions_pred_line = alt.Chart(sessions_preds).mark_line(color = 'black').encode(
    x = 'hour', y='predictions')

sessions_plot = alt.layer(sessions_data_plot, sessions_pred_line).properties(width = 600, height = 200)
sessions_plot

This plot tells us about player activity throughout the day. It seems like higher numbers of active players occur during the day, and is the highest around evening time. 

In [ ]:
dayofweek_plot = alt.Chart(dayofweek_preds).mark_point(opacity = 0.4, filled = True).encode(
    x = alt.X('hour').title('Hour of Day'),
    y = alt.Y('active_sessions').title('Active Sessions'),
    size = 'count():Q'
)

dayofweek_pred_line = alt.Chart(dayofweek_preds).mark_line(color = 'black').encode(
    x = 'hour', y='predictions')

dayofweek_sessions_plot = alt.layer(dayofweek_plot, dayofweek_pred_line).properties(width = 600, height = 100).facet(facet=alt.Facet('day_or_end', title = 'Active Sesssions by Hour for Weekdays and Weekends'),columns = 1)
dayofweek_sessions_plot

These visualizations show us that compared to weekdays, there are more people playing during weekends, and their activity is more spread throughout the day. However, we need to account for the fact that there are five weekday days and only two weekend days, so let's look at the average active sessions.

In [ ]:
avg_sessions_plot = alt.Chart(dayofweek_preds, title = 'Average Active Sessions for Weekdays and Weekends').mark_bar(opacity = 0.8).encode(
    x = alt.X('hour').title('Hour of Day'),
    y = alt.Y('mean(active_sessions)').title('Average Active Sessions').stack(None),
    color = alt.Color('day_or_end', title='Weekday or Weekend').scale(scheme="set1")
)
avg_sessions_plot

In [ ]:
pred_avg_sessions_plot = alt.Chart(dayofweek_preds, title = 'Predicted Average Active Sessions for Weekdays and Weekends').mark_bar(opacity = 0.8).encode(
    x = alt.X('hour').title('Hour of Day'),
    y = alt.Y('mean(predictions)').title('Predicted Average Active Sessions').stack(None),
    color = alt.Color('day_or_end', title='Weekday or Weekend').scale(scheme="set1")
)
pred_avg_sessions_plot

This helps us conclude that there are indeed more active sessions on weekends. More sessions take place throughout the day on weekends, while more sessions during the evenings take place on weekdays. 